In [ ]:
# Necessary Libraries
import os
import pandas as pd
import numpy as np
from googleapiclient.discovery import build
from datetime import datetime
from scipy.stats import gamma
from datetime import timezone
from dateutil import parser
from pathlib import Path

In [ ]:
# Importing the youtube data
file_path = r"C:\Users\...\youtube_data_4K_channels.csv"  #choose your own file path
df = pd.read_csv(file_path)

In [ ]:
# Importing the monte-carlo simulation data
mc_file_path = r"C:\Users\...\mc_simulation_data.csv"  #choose your own file path
sim_df = pd.read_csv(mc_file_path)

In [ ]:
# Constructing the posterior lambda function
def bayesian_lambda_posterior(
    df: pd.DataFrame,
    prior_strength_days: int = 30,
    variability_ratio: float = 0.25
) -> pd.DataFrame:
    """
    Gamma–Poisson Bayesian update for daily growth rate λ per channel.
    We form a prior using per-channel mean growth and variability proxy.

    Prior: λ ~ Gamma(a0, b0)
    Posterior with aggregated evidence: λ ~ Gamma(a_post, b_post)

    - prior_strength_days: pseudo-count of historical days influencing the prior.
    - variability_ratio: used to shape dispersion of prior around mean.
    """
    mean_growth = df["growth_rate_per_day"].to_numpy(dtype=float)
    
    # Construct prior parameters:
    mean_safe = np.clip(mean_growth, 1e-6, None)
    b0 = prior_strength_days / mean_safe
    a0 = mean_safe * b0

    # Aggregate "observed" evidence approximated by current age and mean gains:
    total_days = df["channel_age_days"].to_numpy(dtype=float)
    total_gains = mean_growth * total_days

    # Posterior parameters:
    a_post = a0 + total_gains
    b_post = b0 + total_days

    # Posterior summaries:
    post_mean = a_post / b_post
    post_var = a_post / (b_post ** 2)
    post_std = np.sqrt(post_var)

    out = pd.DataFrame({
        "channel_id": df["channel_id"],
        "lambda_prior_a": a0,
        "lambda_prior_b": b0,
        "lambda_post_a": a_post,
        "lambda_post_b": b_post,
        "lambda_post_mean": post_mean,
        "lambda_post_std": post_std
    })
    return out

In [ ]:
# Constructing the posterior probability function
def bayesian_prob_250k(
    df: pd.DataFrame,
    posterior_df: pd.DataFrame,
    horizon_days: int = 365 * 3,
    sims: int = 5000,
    random_state: int | None = 123
) -> pd.DataFrame:
    """
    Draw λ from posterior Gamma for each channel and simulate horizon gains.
    Compute probability of reaching 250k and credible intervals.
    """

    rng = np.random.default_rng(random_state)

    # Merge safely — only channels with posterior parameters will be included
    merged = df.merge(posterior_df, on="channel_id", how="inner")

    # Extract arrays
    current = merged["current_subscriber_count"].to_numpy(dtype=float)
    a = merged["lambda_post_a"].to_numpy(dtype=float)
    b = merged["lambda_post_b"].to_numpy(dtype=float)

    n = len(merged)  

    # Sample lambda per sim: shape (n_channels, sims)
    lam_samples = rng.gamma(
        shape=a[:, None], 
        scale=1.0 / b[:, None], 
        size=(n, sims)          
    )

    # Predictive gains ~ Poisson(λ * horizon)
    gains = rng.poisson(lam_samples * horizon_days)

    # Ending subscriber count
    ending = current[:, None] + gains

    # Probability of hitting 250k
    hit = ending >= 250_000
    prob = hit.mean(axis=1)

    # Credible intervals
    q10 = np.quantile(ending, 0.10, axis=1)
    q50 = np.quantile(ending, 0.50, axis=1)
    q90 = np.quantile(ending, 0.90, axis=1)

    # Output
    out = pd.DataFrame({
        "channel_id": merged["channel_id"],
        "bayes_prob_250k_in_horizon": prob,
        "bayes_end_subs_q10": q10,
        "bayes_end_subs_q50": q50,
        "bayes_end_subs_q90": q90,
        "horizon_days": horizon_days,
        "sims": sims
    })

    return out

In [ ]:
# Posterior for lambda per channel
posterior_df = bayesian_lambda_posterior(df, prior_strength_days=30, variability_ratio=0.25)
posterior_df.info()

In [ ]:
# Dropping duplicates if any
posterior_df = posterior_df.drop_duplicates(subset="channel_id")
posterior_df.info()

In [ ]:
# Bayesian probabilities of hitting 250k
bayes_res = bayesian_prob_250k(df, posterior_df, horizon_days=365*3, sims=5000)
bayes_res.info()

In [ ]:
# Quick look at the dataframe
bayes_res.head()

In [ ]:
# Quick check on the description
bayes_res['bayes_prob_250k_in_horizon'].describe()

In [ ]:
# Saving the results as a dataframe 
output_dir = Path('C:/Users/.../YT Analysis Data') #choose your own file path
output_filename = 'bayes_estimate_data.csv'
output_filepath = output_dir/output_filename
print(output_filepath)

# Creating the directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Saving the DataFrame to CSV in the new location
bayes_res.to_csv(output_filepath, index=False)
print("Channels Saved Succesfully!")